In [62]:
#papermill_description=imports

import json
import os
import logging
import sys
import geopandas as gpd
from io import StringIO
import pandas as pd
from datetime import datetime
from gis_utils.dataframe import get_bbox_from_geodf
from aws_utils import S3Utils
import calendar
import time
from gis_utils.meteo import OpenMeteoAPI, convert_epoch_to_timezone, map_months_to_numbers
import pytz
from datetime import datetime, timedelta

from aws_utils import S3Utils


logger = logging.getLogger()

In [63]:
#papermill_description=parameters

notebook_key = "localjupyter"

geojson = {
    'body': {
        "type": "FeatureCollection",
        "name": "dissolved-boundaries",
        "crs": {
            "type": "name",
            "properties": {
                "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
            }
        },
        "features": [
            {
                "type": "Feature",
                "properties": {
                    "fid": 1
                },
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [116.26012130269045, -29.225295369642396],
                            [116.261724812149055, -29.241374854584375],
                            [116.283751968396274, -29.256813692452539],
                            [116.284342735038919, -29.268250184258388],
                            [116.292247755352392, -29.265992437426529],
                            [116.292360282331941, -29.293057573630019],
                            [116.314865678242256, -29.293523728033122],
                            [116.326259034921833, -29.293033039128805],
                            [116.326315298411629, -29.305397680579894],
                            [116.355065941687045, -29.307016748931797],
                            [116.355065941687045, -29.306575187382712],
                            [116.383366477044206, -29.307384715430175],
                            [116.384322956370426, -29.290407813444993],
                            [116.387586238777402, -29.282629879611861],
                            [116.386517232471661, -29.259807919053017],
                            [116.359201308185533, -29.259488866292969],
                            [116.359229439930417, -29.259243440415627],
                            [116.35242155766754, -29.259292525638209],
                            [116.352140240218716, -29.220237788279107],
                            [116.302234524787593, -29.223503148505326],
                            [116.281388901825679, -29.2239696200396],
                            [116.26012130269045, -29.225295369642396]
                        ]
                    ]
                },
                "properties": {
                    "boundaryId": "018f9f82-ea97-749d-baea-cd65d3ecc989"
                }
            }
        ]
    }
}
propertyName = "test"
output_type = "weather"

timezone = "Australia/Sydney"

forecast_days = 7

workspaceId = "018f9876-b3d7-73aa-97e6-0cf7a874383d"
propertyId = "018f99ea-564e-72fa-a4b7-2dbd24b65c3e"


In [64]:
#papermill_description=meater_variables_metadata

forecast_hourly = [
		"temperature_2m",
   		"apparent_temperature", # added as apparent temp included in daily forecast
		"relative_humidity_2m",
		"dew_point_2m",
		"precipitation",
		"weather_code",
		"cloud_cover",
		"et0_fao_evapotranspiration",
		"wind_speed_10m",
    	"wind_speed_40m",
    	"wind_direction_10m",
		"wind_direction_40m", # Do we need to fetch data at 10m and 40m above ground?
		"wind_gusts_10m",
		"sunshine_duration", # seconds of sunshine in preceeding hour
		"visibility",
		"soil_temperature_0_to_10cm",
		"soil_moisture_0_to_10cm"
]

forecast_daily = [
		"weather_code",
		"sunrise",
		"sunset",
		"uv_index_max",
		"temperature_2m_max", 
		"temperature_2m_min",
		"apparent_temperature_max", 
		"apparent_temperature_min",
		"daylight_duration",
		"sunshine_duration", # removed uv from forecast as BOM doesn't include it?
		"precipitation_sum",
		"precipitation_hours",
		"wind_speed_10m_max", # added for completeness
		"wind_gusts_10m_max", # added for completeness
		"wind_direction_10m_dominant",
		"shortwave_radiation_sum",
		"et0_fao_evapotranspiration"
]

In [65]:
#papermill_parameters=functions
def process_weather_data(weather_data, variable_order, utc=True):
    """
    Process weather data and return a pandas DataFrame.

    Args:
        weather_data (WeatherData): The weather data object containing the variables.
        variable_order (list): The order of variables to be included in the DataFrame.
        utc (bool, optional): Whether to interpret the time in UTC. Defaults to True.

    Returns:
        pandas.DataFrame: The processed weather data as a DataFrame.

    """
    variables = {
        name: weather_data.Variables(index).ValuesInt64AsNumpy() if name in ['sunrise', 'sunset'] 
              else weather_data.Variables(index).ValuesAsNumpy()
        for index, name in enumerate(variable_order)
    }

    start_time = pd.to_datetime(weather_data.Time(), unit='s', utc=utc)
    end_time = pd.to_datetime(weather_data.TimeEnd(), unit='s', utc=utc)

    time_range = pd.date_range(
        start=start_time,
        end=end_time,
        freq=pd.Timedelta(seconds=weather_data.Interval()),
        inclusive="left"
    )

    # Create a time range if needed
    data = {"date": time_range}
    data.update(variables)

    return pd.DataFrame(data)


def process_forecast(storage_directory, notebook_key, gpd_lat, gpd_lon, boundary_id, forecast_days=15):
    """
    Process the forecast weather for a given location and number of forecaste days, and save the data.
    The maximum number of forecaste days will vary depending on the OpenMeteo model.
    refer to documentation here: https://open-meteo.com/en/docs/bom-api

    Args:
        storage_directory (str): The directory where the forecast data will be stored temporarily before being uploaded to S3.
        notebook_key (str): The key of the notebook.
        gpd_lat (float): The latitude of the location.
        gpd_lon (float): The longitude of the location.
        boundary_id (int): The ID of the boundary.
        forecast_days (int): The number of forecast days. Defaults to 15.

    Returns:
        None
    """

    today = (datetime.now().date()).strftime('%Y-%m-%d')
    
    weather_output_forecast_daily_filename = os.path.join(storage_directory, f"{today}_daily.csv")

    responses = api.fetch_weather_data(
        latitude=gpd_lat,
        longitude=gpd_lon,
        start_date=None,
        end_date=None,
        url="https://api.open-meteo.com/v1/forecast",
        daily=forecast_daily,
        hourly=None,
        timezone=timezone,
        timeformat="unixtime",
        forecast_days=forecast_days
    )
    response = responses[0]

    daily = response.Daily()
    forecast_daily_data = process_weather_data(daily, forecast_daily, utc=True)

    forecast_daily_data = convert_epoch_to_timezone(forecast_daily_data, ["sunrise", "sunset"])

    forecast_daily_data["longitude"] = gpd_lon
    forecast_daily_data["latitude"] = gpd_lat
    forecast_daily_data["boundary_id"] = boundary_id
    forecast_daily_data["boundary_name"] = notebook_key
    forecast_daily_data["workspace_id"] = workspaceId
    forecast_daily_data["property_id"] = propertyId

    forecast_daily_data.to_csv(weather_output_forecast_daily_filename, index=False)

    # This prefix overwrites the feault one set in the cell 'establish_aws_settings'
    prefix=f"weather/forecast/{today}"
    
    s3_client.upload_file(prefix=prefix, file_path=weather_output_forecast_daily_filename)

    time.sleep(1)

In [66]:
#papermill_description=establish_directory

# Set up the initial directory based on the environment
storage_directory = f"/tmp/{notebook_key}"

# Ensure the storage directory exists
os.makedirs(storage_directory, exist_ok=True)

"""
JAG: I've commented out the variables and functions that are for historical data, as we're focusing on getting forecast data as an app for now
"""


"\nJAG: I've commented out the variables and functions that are for historical data, as we're focusing on getting forecast data as an app for now\n"

In [67]:
#papermill_description=establish_aws_settings

#TODO: Check with Nav how to do this right when its pushed to production as well
aws_s3_notebook_output = "notebook-uploads" #sensand-staging-notebook-outputs 
aws_default_region = os.getenv('AWS_DEFAULT_REGION')

s3_client = S3Utils(
		region_name=aws_default_region,
		s3_bucket=aws_s3_notebook_output,
		prefix='weather' #actual prefix is overwritten in the forecast function. Check with Nav abt cleaning this up
)

In [68]:
#papermill_description=processing_file_io

req = geojson
geojson_data = req['body']  # Directly accessing the 'body' since it's already a dictionary in this mock setup

# Convert the GeoJSON string to a GeoDataFrame
gdf = gpd.read_file(StringIO(json.dumps(geojson_data)))

In [69]:
#papermill_description=processing_bounding_box

geom = gdf.geometry #for data-harvester clip function

# Get bounding box from GeoJSON
bbox = get_bbox_from_geodf(geojson_data)

boundary_id = gdf['boundaryId'][0]

gpd_lon = (bbox[0] + bbox[2]) / 2
gpd_lat = (bbox[1] + bbox[3]) / 2

centroid = [gpd_lon, gpd_lat]

In [70]:
#papermill_description=process_forecast_weather_data

api = OpenMeteoAPI()

process_forecast(
    storage_directory=storage_directory,
    notebook_key=notebook_key,
    gpd_lat=gpd_lat,
    gpd_lon=gpd_lon,
    boundary_id=boundary_id,
    forecast_days=forecast_days)


OSError: Cannot save file into a non-existent directory: '/tmp/localjupyter/forecast'